<a href="https://colab.research.google.com/github/ananthu666/Nexlink/blob/main/Nexlink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric

import torch_geometric
print(torch_geometric.__version__)

2.1.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.5 MB/s eta 0:00:00
2.4.0


In [2]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# load the Cora dataset
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.10, num_test=0.20, neg_sampling_ratio = 1.0,
                  is_undirected=True, add_negative_train_samples=False),
])
dataset = Planetoid('./data/Planetoid', name='Cora', transform=transform)
# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.
train_data, val_data, test_data = dataset[0]
print(train_data)
print(val_data)
print(test_data)

Processing...


Data(x=[2708, 1433], edge_index=[2, 7392], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[3696], edge_label_index=[2, 3696])
Data(x=[2708, 1433], edge_index=[2, 7392], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[1054], edge_label_index=[2, 1054])
Data(x=[2708, 1433], edge_index=[2, 8446], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[2110], edge_label_index=[2, 2110])


Done!


In [5]:
(train_data.edge_index[0] <= train_data.edge_index[1]).sum()

tensor(3696)

In [6]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        #return the indices of a non-zero element
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [7]:
model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [8]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1))

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())



In [10]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

Epoch: 001, Loss: 0.4279, Val: 0.9069, Test: 0.9097
Epoch: 002, Loss: 0.4319, Val: 0.9072, Test: 0.9098
Epoch: 003, Loss: 0.4293, Val: 0.9069, Test: 0.9096
Epoch: 004, Loss: 0.4333, Val: 0.9072, Test: 0.9104
Epoch: 005, Loss: 0.4285, Val: 0.9077, Test: 0.9107
Epoch: 006, Loss: 0.4220, Val: 0.9075, Test: 0.9097
Epoch: 007, Loss: 0.4245, Val: 0.9078, Test: 0.9095
Epoch: 008, Loss: 0.4267, Val: 0.9079, Test: 0.9093
Epoch: 009, Loss: 0.4188, Val: 0.9076, Test: 0.9091
Epoch: 010, Loss: 0.4252, Val: 0.9077, Test: 0.9100
Epoch: 011, Loss: 0.4212, Val: 0.9082, Test: 0.9103
Epoch: 012, Loss: 0.4232, Val: 0.9090, Test: 0.9102
Epoch: 013, Loss: 0.4220, Val: 0.9097, Test: 0.9104
Epoch: 014, Loss: 0.4241, Val: 0.9090, Test: 0.9101
Epoch: 015, Loss: 0.4299, Val: 0.9077, Test: 0.9093
Epoch: 016, Loss: 0.4239, Val: 0.9085, Test: 0.9097
Epoch: 017, Loss: 0.4190, Val: 0.9098, Test: 0.9095
Epoch: 018, Loss: 0.4198, Val: 0.9096, Test: 0.9095
Epoch: 019, Loss: 0.4137, Val: 0.9101, Test: 0.9106
Epoch: 020, 

In [13]:
z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

In [14]:
final_edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    1,    2,  ..., 2705, 2706, 2707]])